# Preprocessing Steps for Light Beads Microscopy Data

### Steps
1. Load raw .tif file
2. Pull ScanImage ROI's and Metadata
3. Restructure Planes
4. Save processed data

sorts the z-planes (LBM)
uses metadata to place MROIs and reconstruct the frames
calculates and corrects the MROI overlaps and seams
calculates and corrects the X-Y shifts across planes (LBM)
outputs data as t-x-y planes or t-x-y-z volumes
saves a .png with the average frame of all planes
saves a short .mp4 clip

In [3]:
import copy
import cv2
import datetime
import glob
import h5py
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy
import scipy.signal
import skimage
import sys
import tifffile
import time
from pathlib import Path

mod_path = Path(os.getcwd()) / '..'
if str(mod_path) not in sys.path:
    sys.path.append(str(mod_path))
import params
plt.rcParams["figure.dpi"] = 900

try:
    from icecream import ic, install, argumentToString
    install()
except ImportError:  # graceful fallback if icecream isn't installed.
    ic = lambda *a: None if not a else (a[0] if len(a) == 1 else a)  # noqa

@argumentToString.register(np.ndarray)
def _(obj):
    """Format our return for the icecream debug statement."""
    return f"ndarray, shape={obj.shape}, dtype={obj.dtype}"

params = params.init_params()

if params['debug']:
    ic.enable()
    ic.configureOutput(prefix='RBO Debugger -> ', includeContext=True, contextAbsPath=True)
    ic()
else:
    ic.disable()
    ic()


RBO Debugger -> /tmp/ipykernel_127276/257540432.py:41 in <module> at 14:40:34.431


In [4]:
if params["save_output"]:
    # 'planes' will save individual planes in subfolders -- 'volume' will save a whole 4D hdf5 volume
    params["save_as_volume_or_planes"] = "planes"
    if params["save_as_volume_or_planes"] == "planes":
        # If True, it will take all the time-chunked h5 files, concatenate, and save them as a single .tif
        params["concatenate_all_h5_to_tif"] = False

if params["seams_overlap"] == "calculate":
    # correct delay or incorrect phase when EOM turns the laser on/off at the start/end of a resonant-scanner line
    params["n_ignored_pixels_sides"] = 5
    params["min_seam_overlap"] = 5
    params["max_seam_overlap"] = 20
    params["alignment_plot_checks"] = False
if not params["reconstruct_all_files"]:
    params["reconstruct_until_this_ifile"] = 10
if params["save_mp4"] or params["save_meanf_png"]:
    params["gaps_columns"] = 5
    params["gaps_rows"] = 5
    params["intensity_percentiles"] = [15, 99.5]
    if params["save_meanf_png"]:
        params["meanf_png_only_first_file"] = True
    if params["save_mp4"]:
        params["video_only_first_file"] = True
        params["video_play_speed"] = 1
        params["rolling_average_frames"] = 1
        params["video_duration_secs"] = 20
        
if not params["lateral_align_planes"]:
    initialize_volume_with_nans = False
    convert_volume_float32_to_int16 = True
    params["make_nonan_volume"] = False
elif params["make_nonan_volume"]:
    initialize_volume_with_nans = True
    convert_volume_float32_to_int16 = True
else:
    initialize_volume_with_nans = True
    convert_volume_float32_to_int16 = False

Retrieve files from the given data directory, any files that match `.tif`, recursively.

In [5]:
# %% Look for files used to: 1) make a template and do seam-overlap handling and X-Y shift alignment; 2) pre-process
path_all_files = []
for i_dir in params["raw_data_dirs"]:
    tmp_paths = sorted(glob.glob(i_dir + "/**/*.tif", recursive=True))
    for this_tmp_path in tmp_paths:
        if (
                params["fname_must_contain"] in this_tmp_path and
                params["fname_must_NOT_contain"] not in this_tmp_path
        ):
            path_all_files.append(this_tmp_path)

if params["debug"]:
    ic(path_all_files)

n_template_files = len(params["list_files_for_template"])
ic(n_template_files)
path_template_files = [path_all_files[file_idx] for file_idx in params["list_files_for_template"]]

del (
    i_dir,
    params["raw_data_dirs"],
    params["fname_must_contain"],
    params["fname_must_NOT_contain"],
)

pipeline_steps = []
if params["make_template_seams_and_plane_alignment"]:
    pipeline_steps.append("make_template")
if params["reconstruct_all_files"]:
    pipeline_steps.append("reconstruct_all")


RBO Debugger -> /tmp/ipykernel_127276/934981101.py:13 in <module>
                path_all_files: ['/v-data4/foconnell/data/lbm/raw/mh89_hemisphere_FOV_50_550um_depth_250mW_dual_stimuli_30min_00001_00001.tif']
RBO Debugger -> /tmp/ipykernel_127276/934981101.py:16 in <module>
                n_template_files: 1


In [6]:
for current_pipeline_step in pipeline_steps:
    if current_pipeline_step == "make_template":
        path_input_files = path_template_files
    elif current_pipeline_step == "reconstruct_all":
        path_input_files = path_all_files

    if params["reconstruct_all_files"]:
        list_files_for_reconstruction = range(len(path_input_files))
    else:
        list_files_for_reconstruction = range(params["reconstruct_until_this_ifile"])

    for i_file in list_files_for_reconstruction:
        tic = time.time()
        path_input_file = path_input_files[i_file]
        ic("Start Reconstruction", path_input_file)
        
        if i_file == 0:
            if "SP" in path_input_file:
                n_planes = 1
            elif "Max15" in path_input_file:
                n_planes = 15
            elif "Max30" in path_input_file:
                n_planes = 30
            elif params["flynn_temp_param"]:
                n_planes = 30
            else:
                n_planes = int(input("Check filename... Number of planes?"))

            if n_planes == 1:
                chans_order = params["chans_order_1plane"]
                rows, columns = 1, 1  # For png and mp4
            elif n_planes == 15:
                chans_order = params["chans_order_15planes"]
                rows, columns = 3, 5
            elif n_planes == 30:
                chans_order = params["chans_order_30planes"]
                rows, columns = 6, 5

            ic(n_planes)

            # %% Get MROI info from tif metadata
            with tifffile.TiffFile(path_input_file) as tif:
                metadata = {}
                for tag in tif.pages[0].tags.values():
                    tag_name, tag_value = tag.name, tag.value
                    metadata[tag_name] = tag_value

            mrois_si_raw = json.loads(metadata["Artist"])["RoiGroups"]["imagingRoiGroup"]["rois"]
            if type(mrois_si_raw) != dict:
                mrois_si = []
                for roi in mrois_si_raw:
                    if type(roi["scanfields"]) != list:
                        scanfield = roi["scanfields"]
                    else:
                        scanfield = roi["scanfields"][
                            np.where(np.array(roi["zs"]) == 0)[0][0]
                        ]
                    roi_dict = {}
                    roi_dict["center"] = np.array(scanfield["centerXY"])
                    roi_dict["sizeXY"] = np.array(scanfield["sizeXY"])
                    roi_dict["pixXY"] = np.array(scanfield["pixelResolutionXY"])
                    mrois_si.append(roi_dict)
            else:
                scanfield = mrois_si_raw["scanfields"]
                roi_dict = {}
                roi_dict["center"] = np.array(scanfield["centerXY"])
                roi_dict["sizeXY"] = np.array(scanfield["sizeXY"])
                roi_dict["pixXY"] = np.array(scanfield["pixelResolutionXY"])
                mrois_si = [roi_dict]

            # Sort MROIs so they go from left-to-right (but keep the un-sorted because that matches how they were acquired and saved in the long-tif-strip)
            mrois_centers_si = np.array([mroi_si["center"] for mroi_si in mrois_si])
            x_sorted = np.argsort(mrois_centers_si[:, 0])
            mrois_si_sorted_x = [mrois_si[i] for i in x_sorted]
            mrois_centers_si_sorted_x = [mrois_centers_si[i] for i in x_sorted]
        # ----
        # %% Load, reshape (so time and planes are 2 independent dimensions) and re-order (planes, fix Jeff's order)
        ic("Loading file (expect warning for multi-file recording)")

        tiff_file = tifffile.imread(path_input_file)
        dim1 = tiff_file.shape[0]
        dim2 = tiff_file.shape[1]
        dim3 = tiff_file.shape[2]

        tiff_file = tifffile.imread(path_input_file)
        
        if n_planes > 1: 
            tiff_file = np.reshape(tiff_file, ( int(tiff_file.shape[0]/n_planes), n_planes,tiff_file.shape[1],tiff_file.shape[2]), order='A') # warnings are expected if the recording is split into many files or incomplete
        else:
            tiff_file = np.expand_dims(tiff_file,1)
        tiff_file = np.swapaxes(tiff_file, 1, 3)
        tiff_file = tiff_file[...,chans_order]
        
        if current_pipeline_step == 'make_template':
            tiff_file = np.mean(tiff_file, axis = 0, keepdims = True)
        
        #%% Separate tif into MROIs
        
        # Get the Y coordinates for mrois (and not flybacks)
        if i_file == 0:
            n_mrois = len(mrois_si)
            tif_pixels_Y = tiff_file.shape[2]
            mrois_pixels_Y = np.array([mroi_si['pixXY'][1] for mroi_si in mrois_si])
            each_flyback_pixels_Y = (tif_pixels_Y - mrois_pixels_Y.sum())//(n_mrois - 1)
        
        if params['json_logging']: json_logger.debug(json.dumps({'debug_message':'Separating tif into individual MROIs'}))
        
        # Divide long stripe into mrois
        planes_mrois = np.empty((n_planes,n_mrois), dtype=np.ndarray)
        for i_plane in range(n_planes):
            y_start = 0
            for i_mroi in range(n_mrois): # We go over the order in which they were acquired
                planes_mrois[i_plane, i_mroi] = tiff_file[:, :, y_start:y_start+mrois_pixels_Y[x_sorted[i_mroi]], i_plane]
                y_start += mrois_pixels_Y[i_mroi] + each_flyback_pixels_Y
        
        del tiff_file


RBO Debugger -> /tmp/ipykernel_127276/2386270191.py:15 in <module>
                "Start Reconstruction": 'Start Reconstruction'
                path_input_file: '/v-data4/foconnell/data/lbm/raw/mh89_hemisphere_FOV_50_550um_depth_250mW_dual_stimuli_30min_00001_00001.tif'
RBO Debugger -> /tmp/ipykernel_127276/2386270191.py:39 in <module>
                n_planes: 30
RBO Debugger -> /tmp/ipykernel_127276/2386270191.py:78 in <module>
                "Loading file (expect warning for multi-file recording)": 'Loading file (expect warning for multi-file recording)'
<tifffile.TiffFile 'mh89_hemisphere…_00001_00001.tif'> asarray failed to reshape (25320, 5104, 145) to (8440, 30, 5104, 145)
RBO Debugger -> /tmp/ipykernel_127276/2386270191.py:15 in <module>
                "Start Reconstruction": 'Start Reconstruction'
                path_input_file: '/v-data4/foconnell/data/lbm/raw/mh89_hemisphere_FOV_50_550um_depth_250mW_dual_stimuli_30min_00001_00001.tif'
RBO Debugger -> /tmp/ipykernel_1272

In [7]:
#   #   #
        

TypeError: 'module' object is not callable